In [20]:
import os
import glob
from datetime import date

import pandas as pd
import numpy as np
import re
from scipy.spatial.transform import Rotation

import matplotlib.pyplot as plt
import matplotlib.pylab as mpl
import seaborn as sns

from IPython.display import display

from functools import partial

from utils.multi_processing_functions import *

In [3]:
DATA_PATH = '../Data/'
META_DATA_PATH = '../Data/experiment_meta/'
PLOT_PATH = '../Plots/'

sns.set(palette='muted', context='talk', style='ticks')
scpt = sns.color_palette()

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [4]:
import multiprocessing as mp

def pandas_mp_apply(grouped_df, apply_func):
    with mp.Pool(mp.cpu_count() - 1) as p:
        ret_list = p.map(apply_func, [(group, name) for name, group in grouped_df])
    return pd.concat(ret_list)

# Preprocessing

## Object dataframe

In [5]:
all_files = glob.glob(os.path.join(META_DATA_PATH, '*.csv'))

objects_list = []

for filename in all_files:
    df = pd.read_csv(filename, sep=';', index_col=False)
    
    identifier = re.split('/|_', filename)[-3]
    df['object'] = identifier.capitalize() 
    
    objects_list.append(df)

objects_df = pd.concat(objects_list, axis=0, ignore_index=True)

objects_df = objects_df.rename(columns = {
    'gridPosition': 'grid_position',
    'world position' : 'world_position',
})

# remove braces and rewrite grid prositions (e.g. (1, 2) to 1_2)
objects_df['grid_position'] = objects_df['grid_position'].str[1:-1]
objects_df['grid_position'] = objects_df['grid_position'].replace(regex=', ', value='_')

# the grid position starts with zero -> increment grid positions
# e.g. 0_0 to 1_1, 4_3 to 5_4, etc.
pos = ['5', '4', '3', '2', '1', '0']

for i in range(1,6):
    objects_df['grid_position'] = objects_df['grid_position'].replace(regex=pos[i], value=pos[i-1])

# remove braces and rewrite world positions (e.g. (-0,882, 1,926, 0,211) to -0.882, 1.926, 0.211)   
objects_df['world_position'] = objects_df['world_position'].str[1:-1]
objects_df['world_position'] = objects_df['world_position'].replace(regex=',', value='.')
objects_df['world_position'] = objects_df['world_position'].replace(regex='. ', value=', ')  

# extract the single world positions and add it to objects_df
world_pos_df = objects_df['world_position'].str.split(', ', expand=True).astype('float')

world_pos_df = world_pos_df.rename(columns = {
    0: 'world_position_x',
    1: 'world_position_y',
    2: 'world_position_z',
})

objects_df = pd.concat([objects_df, world_pos_df], axis=1)
#objects_df.to_csv(f'{DATA_PATH}02-1_objects.csv', index=False)
display(objects_df.head())

,grid_position,world_position,object,world_position_x,world_position_y,world_position_z
0,1_1,"-0.882, 1.926, 0.211",Meta\cube,-0.882,1.926,0.211
1,2_1,"-0.436, 1.926, 0.211",Meta\cube,-0.436,1.926,0.211
2,3_1,"0.010, 1.926, 0.211",Meta\cube,0.010,1.926,0.211
3,4_1,"0.456, 1.926, 0.211",Meta\cube,0.456,1.926,0.211
4,5_1,"0.902, 1.926, 0.211",Meta\cube,0.902,1.926,0.211


## General samples dataframe

In [6]:
DATA_PATH_2 = 'D:/projects/ErgoVR/processed_data/EIH_1/'
samples_df = pd.read_csv(f'{DATA_PATH_2}06_ET_cleaned_EHC.csv')

C:\Users\akeshava\.conda\envs\conda_ergovr\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (69,70) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:


samples_df = samples_df.rename(columns = {
    'handData_transform_position_x' : 'hand_position_x',
    'handData_transform_position_y' : 'hand_position_y',
    'handData_transform_position_z' : 'hand_position_z',
})

samples_df['trial_id'] = samples_df['trial_id'].fillna(0)
samples_df.trial_id = samples_df.trial_id.astype(int)

samples_df = (
    samples_df
    .query('eye_shelf_hit != "ERROR"')
    .query('subject_id != 2022 and trial_num != 0')
    .query('subject_id != 2044 and trial_num != 1')
)

samples_df.loc[
    (
        (samples_df.subject_id == 2040)
        & (samples_df.subjectfileName.str.endswith('_2.etd'))
    ), 'trial_num'
] += 3

samples_df['trial_type'] = (
    samples_df
    .trial_id
    .apply(lambda s: 'easy' if s in np.arange(0,8) else 'hard')
)

valid_objs = ['Cube_Blue', 'Cube_Green', 'Cube_Red', 'Cube_Yellow',
    'Cylinder_Blue', 'Cylinder_Green', 'Cylinder_Red','Cylinder_Yellow',
    'Sphere_Blue', 'Sphere_Green', 'Sphere_Red', 'Sphere_Yellow', 
    'Tetraeder_Blue', 'Tetraeder_Green', 'Tetraeder_Red', 'Tetraeder_Yellow']

samples_df.loc[~(samples_df.eye_hit.isin(valid_objs)) & 
           ~(pd.isnull(samples_df.eye_hit)),
               'eye_hit'] ='Other'

samples_df.dropna(subset = ['hand_position_z'], inplace=True)
samples_df.reset_index(drop=True, inplace=True)

#samples_df.to_csv(f'{DATA_PATH}02-2_samples.csv', index=False)
display(samples_df.head())

,timestamp_dt,combinedEye_position_x,combinedEye_position_y,combinedEye_position_z,combinedEye_direction_x,combinedEye_direction_y,combinedEye_direction_z,eye_hit,combinedEye_raycastHitLocation_x,combinedEye_raycastHitLocation_y,...,fix_onset_bool,fix_stop_bool,grasp_onset,grasp_stop,grasp_duration,grasp_onset_bool,grasp_end_bool,pickup_location,drop_location,trial_type
0,1970-01-01 00:00:00.351460844,-0.009295,-0.002215,-0.023014,0.038575,0.107453,0.993462,Other,0.227122,1.447578,...,True,False,1970-01-01 00:00:00.351460844,1970-01-01 00:00:00.351460844,0.0,False,False,NaN,NaN,hard
1,1970-01-01 00:00:00.360775739,-0.004848,-0.001775,-0.023124,0.037461,0.100694,0.994212,Other,0.225640,1.438154,...,False,False,1970-01-01 00:00:00.360775739,1970-01-01 00:00:00.360775739,0.0,False,False,NaN,NaN,hard
2,1970-01-01 00:00:00.371934354,-0.004848,-0.001775,-0.023124,0.037461,0.100694,0.994212,Other,0.224285,1.438936,...,False,False,1970-01-01 00:00:00.371934354,1970-01-01 00:00:00.371934354,0.0,False,False,NaN,NaN,hard
3,1970-01-01 00:00:00.383572042,-0.009973,-0.002281,-0.022997,0.037522,0.107240,0.993525,Other,0.222477,1.448279,...,False,False,1970-01-01 00:00:00.383572042,1970-01-01 00:00:00.383572042,0.0,False,False,NaN,NaN,hard
4,1970-01-01 00:00:00.394215554,0.004572,-0.000881,-0.023263,0.042648,0.104645,0.993595,Other,0.228530,1.445886,...,False,False,1970-01-01 00:00:00.394215554,1970-01-01 00:00:00.394215554,0.0,False,False,NaN,NaN,hard


## General samples dataframe with angles

In [9]:
def calculate_angles_body_local(hand_pos1, hand_pos2, head_pos1, head_pos2, head_dir1, head_dir2):
    
    hand_pos = np.asarray([hand_pos1, hand_pos2])
    head_pos = np.asarray([head_pos1, head_pos2])
    head_dir = np.asarray([head_dir1, head_dir2])
    
    hand_unit_vec = (hand_pos-head_pos) / np.linalg.norm(hand_pos-head_pos)
    head_unit_vec = head_dir / np.linalg.norm(head_dir)
    
    angle = np.arccos(np.dot(hand_unit_vec, head_unit_vec))
    
    if hand_pos1<head_pos1:
        angle = -angle
    
    return np.degrees(angle)

In [10]:
angles_df = samples_df.copy()

#calculate angles for hand (only local)
angles_df['hand_local_theta_h'] = angles_df.apply(
                 lambda x: calculate_angles_body_local(
                     x['hand_position_x'], x['hand_position_z'], x['nosePointer_position_x'], 
                     x['nosePointer_position_z'], x['nosePointer_direction_x'], x['nosePointer_direction_z']
                 ), axis=1
             )

angles_df['hand_local_theta_v'] = angles_df.apply(
                 lambda x: calculate_angles_body_local(
                     x['hand_position_y'], x['hand_position_z'], x['nosePointer_position_y'], 
                     x['nosePointer_position_z'], x['nosePointer_direction_y'], x['nosePointer_direction_z']
                 ), axis=1
             )

In [11]:
#calculate angles for eye (local and global)
angles_df['eye_local_theta_h']  = np.degrees(np.arctan2(angles_df['combinedEye_direction_x'], angles_df['combinedEye_direction_z']))
angles_df['eye_local_theta_v']  = np.degrees(np.arctan2(angles_df['combinedEye_direction_y'], angles_df['combinedEye_direction_z'])) 

angles_df['eye_global_theta_h'] = np.degrees(np.arctan2(angles_df['EIH_dir_x'], angles_df['EIH_dir_z']))
angles_df['eye_global_theta_v'] = np.degrees(np.arctan2(angles_df['EIH_dir_y'], angles_df['EIH_dir_z']))

In [12]:
angles_df.head()

#save dataset in degree
angles_df.to_csv(f'{DATA_PATH}02-3_samples_angles_deg.csv', index=False)

In [13]:
del angles_df
del samples_df

## Samples dataframe with angles (eye)

In [16]:
samples_df = pd.read_csv(f'{DATA_PATH}02-3_samples_angles_deg.csv')

samples_df[['timestamp_dt','fix_onset','fix_stop', 'grasp_onset', 'grasp_stop']] = samples_df[[
    'timestamp_dt','fix_onset','fix_stop', 'grasp_onset', 'grasp_stop']].apply(pd.to_datetime)

samples_eye_df = samples_df[[
    'subject_id', 'trial_id', 'trial_num', 'trial_type', 'timestamp_dt', 'is_fixation', 'eye_hit', 
    'grasp', 'grasp_onset_bool', 'eye_shelf_hit', 'fix_duration', 'grasp_end_bool', 'grasp_duration', 
    'is_outlier_fixation', 'pickup_location', 'drop_location', 'hand_position_x', 'hand_position_y', 
    'hand_position_z', 'nosePointer_position_x', 'nosePointer_position_y', 'nosePointer_position_z',
    'hand_local_theta_h', 'hand_local_theta_v', 'eye_local_theta_h', 'eye_local_theta_v', 
    'eye_global_theta_h', 'eye_global_theta_v',
]]

# process data
# in seconds
bin_size = 0.2
offset_start = -2
offset_stop = 2

grp_cols = ['subject_id', 'trial_num']
grasp_epochs_eye_df = (
    pandas_mp_apply(
        samples_eye_df.groupby(grp_cols),
        partial(get_epoch_grasp_on_eye, objects_df=objects_df, offset_start=offset_start, offset_stop=offset_stop)
    )
)
grasp_epochs_eye_df.head()

grasp_epochs_eye_df.to_csv(f'{DATA_PATH}02-4_grasp_epoch_eye.csv', index=True)

## Samples dataframe with angles (hand) - pickup

!!! contains only usable trials (see check_usable_trials for reference) !!!

In [25]:
samples_hand_df = samples_df[[
    'subject_id', 'trial_id', 'trial_num', 'trial_type', 'timestamp_dt', 'is_fixation', 'eye_hit', 
    'grasp', 'grasp_onset_bool', 'eye_shelf_hit', 'fix_duration', 'grasp_end_bool', 'grasp_duration', 
    'is_outlier_fixation', 'pickup_location', 'drop_location', 'hand_position_x', 'hand_position_y', 
    'hand_position_z', 'nosePointer_position_x', 'nosePointer_position_y', 'nosePointer_position_z',
    'hand_local_theta_h', 'hand_local_theta_v', 'eye_local_theta_h', 'eye_local_theta_v', 
    'eye_global_theta_h', 'eye_global_theta_v',
]]

unusable = [2013, 2020, 2023, 2024, 2032, 2033, 2037, 2038, 2040, 2041, 2042, 2045, 2047, 2050, 
            2054, 2055, 2056, 2057, 2058, 2060, 2061, 2062]

samples_subset_df = samples_hand_df.copy()
samples_subset_df = samples_subset_df.query('subject_id not in @unusable')

# keep trial numbers 2-5
samples_subset_df.drop(samples_subset_df[(samples_subset_df['subject_id'].isin([2007, 2009, 2019])) & (samples_subset_df['trial_num'] > 5)].index, inplace=True)
# keep trial numbers 2-11
samples_subset_df.drop(samples_subset_df[(samples_subset_df['subject_id'].isin([2026, 2030])) & (samples_subset_df['trial_num'] > 11)].index, inplace=True)
# keep trial numbers 2-17
samples_subset_df.drop(samples_subset_df[(samples_subset_df['subject_id'].isin([2046])) & (samples_subset_df['trial_num'] > 17)].index, inplace=True)
# keep trial numbers 12-16
samples_subset_df.drop(samples_subset_df[(samples_subset_df['subject_id'].isin([2039])) & ((samples_subset_df['trial_num'] < 12) | (samples_subset_df['trial_num'] > 16))].index, inplace=True)
# keep trial numbers 12-23
samples_subset_df.drop(samples_subset_df[(samples_subset_df['subject_id'].isin([2008, 2010, 2014, 2016, 2018, 2025])) & (samples_subset_df['trial_num'] < 12)].index, inplace=True)
# keep trial numbers 16-23
samples_subset_df.drop(samples_subset_df[(samples_subset_df['subject_id'].isin([2034])) & (samples_subset_df['trial_num'] < 16)].index, inplace=True)

# process data
# in seconds
bin_size = 0.2
offset_start = -2
offset_stop = 2

grp_cols = ['subject_id', 'trial_num']
grasp_epochs_body_df = (
    pandas_mp_apply(
        samples_subset_df.groupby(grp_cols),
        partial(get_epoch_grasp_on_body, 
                objects_df=objects_df, 
                offset_start=offset_start, 
                offset_stop=offset_stop, 
                bin_size=bin_size)
    )
)

grasp_epochs_body_df.to_csv(f'{DATA_PATH}02-5_grasp_epoch_body.csv', index=True)

In [26]:
grasp_epochs_body_df

trial_type                  timestamp_dt  is_fixation  \
subject_id trial_num                                                         
2006.0     2.0             hard 1970-01-01 00:01:14.080726624         True   
           2.0             hard 1970-01-01 00:01:14.091705322         True   
           2.0             hard 1970-01-01 00:01:14.103096008         True   
           2.0             hard 1970-01-01 00:01:14.114341736         True   
           2.0             hard 1970-01-01 00:01:14.125267029         True   
...                         ...                           ...          ...   
2059.0     23.0            easy 1970-01-01 00:00:45.503601074        False   
           23.0            easy 1970-01-01 00:00:45.514438629        False   
           23.0            easy 1970-01-01 00:00:45.525497436        False   
           23.0            easy 1970-01-01 00:00:45.537063599        False   
           23.0            easy 1970-01-01 00:00:45.547912598        False   

                      hand_position_x  hand_position_y  hand_position_z  \
subject_id trial_num                                                      
2006.0     2.0               0.217474         0.719120        -1.186495   
           2.0               0.214988         0.716656        -1.183078   
           2.0               0.213497         0.715411        -1.178555   
           2.0               0.211275         0.713802        -1.173440   
           2.0               0.209265         0.712598        -1.167058   
...                               ...              ...              ...   
2059.0     23.0              0.321762         0.715217        -0.487616   
           23.0              0.330811         0.717943        -0.502123   
           23.0              0.340079         0.720769        -0.516580   
           23.0              0.348834         0.723285        -0.531255   
           23.0              0.357576         0.725608        -0.545977   

                      nosePointer_position_x  nosePointer_position_y  \
subject_id trial_num                                                   
2006.0     2.0                     -0.174134                1.541287   
           2.0                     -0.174876                1.541364   
           2.0                     -0.175707                1.541392   
           2.0                     -0.176456                1.541430   
           2.0                     -0.177175                1.541516   
...                                      ...                     ...   
2059.0     23.0                    -0.033716                1.547560   
           23.0                    -0.032182                1.550859   
           23.0                    -0.030762                1.554185   
           23.0                    -0.029176                1.557948   
           23.0                    -0.027686                1.561868   

                      nosePointer_position_z  hand_local_theta_h  ...  \
subject_id trial_num                                              ...   
2006.0     2.0                     -1.159859          108.011055  ...   
           2.0                     -1.154882          108.470792  ...   
           2.0                     -1.149731          108.771793  ...   
           2.0                     -1.144598          108.993270  ...   
           2.0                     -1.139277          109.032662  ...   
...                                      ...                 ...  ...   
2059.0     23.0                    -0.692094           70.845269  ...   
           23.0                    -0.702048           70.941218  ...   
           23.0                    -0.711899           71.173606  ...   
           23.0                    -0.721416           71.584603  ...   
           23.0                    -0.730638           72.089635  ...   

                      eye_global_theta_h  eye_global_theta_v  grasp_num  \
subject_id trial_num                                                